In [68]:
import numpy as np
import random
from random import randint
import collections
from copy import deepcopy
from Chromosome import *
import math

In [69]:
#---------------------------------------------------------------#
TOTAL_POPULATION = 50
MAX_GENERATIONS = 500
CROSS_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.05
STUDENT_GROUPS_GLOBAL = {}
TEACHERS_GLOBAL = []
COURSES_GLOBAL = []
CLASS_IDS_GLOBAL = []
STUDENT_LIST_GLOBAL = []
#---------------------------------------------------------------#


In [70]:
def main():
    file_name = './studentCourse.csv'
    courses_code_list, student_groups, teachers_list, class_IDs, student_names_list = create_Datasets(file_name)
    global STUDENT_GROUPS_GLOBAL
    global TEACHERS_GLOBAL
    global COURSES_GLOBAL
    global CLASS_IDS_GLOBAL
    global STUDENT_LIST_GLOBAL
    STUDENT_GROUPS_GLOBAL = student_groups
    TEACHERS_GLOBAL = deepcopy(teachers_list)
    COURSES_GLOBAL = deepcopy(courses_code_list)
    CLASS_IDS_GLOBAL = deepcopy(class_IDs)
    STUDENT_LIST_GLOBAL = deepcopy(student_names_list)
    population = []
    for i in range(TOTAL_POPULATION):
        population.append(populate(courses_code_list,
                                   student_groups, teachers_list, class_IDs))
        population[i].index = i
        population[i].chromo_To_Bits(TOTAL_POPULATION)
        print("CHROMO {} = {}\n\n".format(i, population[i].chromoBits))
        if i == 99:
            print("Teachers Size: ", len(
                population[i].days[0].sessions[0].teachers) + len(population[i].days[0].sessions[1].teachers))
    
    GA(population)

In [83]:
def calculate_fitness(population):
    # print("Fitness Calc")
    ### Your Code Here ####
    global STUDENT_GROUPS_GLOBAL
    global TEACHERS_GLOBAL
    global COURSES_GLOBAL
    global STUDENT_LIST_GLOBAL
    # fitness_values = []

    # Value of fitness in terms of exams per day
    relativity = [0.1, 0.1, 0.5, 0.75, 0.8, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01]
    # relativity = [0.07, 0.5, 0.5, 0.6, 0.6,
    #               0.4, 0.3, 0.001, 0.001, 0.001, 0.001]

    # -> Overlapping Students, Overlapping Teachers, Consecutive Teachers, Duplicate Exams
    # -> 100                   10                    10                    100

    for i, chromo in enumerate(population):
        overlap_student = 0
        overlap_teacher = 0
        consecutive_Students = False
        consecutive_Teachers = 0
        weekend_days = 0
        population[i].remove_Empty_Days()
        chromo.remove_Empty_Days()
        FITNESS_CONSTANT = 100000.0
        csBeforemg = 0
        # total_Conflicts = 0
        for j, day in enumerate(chromo.days):
            for session in day.sessions:

                overlap = session.overlapping_Students1()
                # print("Students overlap:", overlap)
                if overlap == True:
                    overlap_student += 2

                overlap = session.overlapping_Teachers()
                if overlap > 0:
                    # print("T")
                    overlap_teacher += overlap

                if (j+1) % 9 == 0 or (j+1) % 10 == 0:
                    if session.courseIDs is not None:
                        weekend_days += 2
            overlap = day.consecutive_Students()
            if overlap > 0:
                consecutive_Students += 2

            overlap = day.consecutive_Teachers()
            if overlap > 0:
                consecutive_Teachers += 5

        duplicate_Exams = chromo.duplicate_Exams()
        total_Exams = chromo.total_exams()
        total_Days = chromo.days_count
        remaining_Exams = len(COURSES_GLOBAL) - total_Exams + duplicate_Exams

        # print("Rems: ", remaining_Exams)
        if chromo.CS_before_MG() == False:
            # print("CS before MG")
            population[i].soft["MG before CS"] = "Not Violated"

        else:
            FITNESS_CONSTANT -= 5
            population[i].soft["MG before CS"] = "Violated"
        if overlap_student > 0:
            population[i].hard["Students Overlap"] = "Violated"
        if overlap_student > 6:
            # print("OS")
            FITNESS_CONSTANT -= overlap_student*10
        else:
            population[i].hard["Students Overlap"] = "Not Violated"

        if overlap_teacher > 0:
            # print("ttt")
            population[i].hard["Teachers Overlap"] = "Violated"
            FITNESS_CONSTANT -= 10
        else:
            population[i].hard["Teachers Overlap"] = "Not Violated"

        if weekend_days > 0:
            population[i].hard["Weekend"] = "Violated"
            FITNESS_CONSTANT -= weekend_days*30
        else:
            population[i].hard["Weekend"] = "Not Violated"

        if consecutive_Students > 0:
            population[i].soft["Consecutive Exam"] = "Violated"
            FITNESS_CONSTANT -= consecutive_Students
        else:
            population[i].soft["Consecutive Exam"] = "Not Violated"

        if consecutive_Teachers > 0:
            # print(" c t")
            population[i].hard["Consecutive Teachers"] = "Violated"
            FITNESS_CONSTANT -= 10
        else:
            population[i].hard["Consecutive Teachers"] = "Not Violated"
        if duplicate_Exams > 0:
            # print("dup")
            population[i].hard["Duplicate Course"] = "Violated"
            FITNESS_CONSTANT -= 80*duplicate_Exams
        else:
            population[i].hard["Duplicate Course"] = "Not Violated"
        if remaining_Exams > 0:
            # print("r")
            population[i].hard["All Courses"] = "Violated"
            FITNESS_CONSTANT -= 80*remaining_Exams
        else:
            population[i].hard["All Courses"] = "Not Violated"
        if total_Days >= (len(COURSES_GLOBAL)/2) :
            FITNESS_CONSTANT -= total_Days*20
        # for day in chromo.days:
        #     FITNESS_CONSTANT *= relativity[day.total_exams]
        population[i].fitness = FITNESS_CONSTANT
        # print("Chromo {} with fitness {}".format(i, FITNESS_CONSTANT))
    for i, chromo in enumerate(population):
        population[i].index = i
    return population

In [72]:
def roulette_wheel_selection(population):
    # print("Wheel Sel")

    # Sum of fitnesses of population
    population_fitness = sum(
        [chromosome.fitness for chromosome in population])
    # pick = random.uniform(0, population_fitness)
    # Probability = Chromo / (sum of fitnesses)
    chromosome_probabilities = [chromosome.fitness /
                                population_fitness for chromosome in population]

    # Selects one chromosome randomly given the probabilities
    chromosome = np.random.choice(population, p=chromosome_probabilities)

    # range_list = []
    # range_list.append(0.0)
    # for prob in chromosome_probabilities:
    #     range_list.append(float(range_list[-1]+prob))
    # rand_num = random.random()
    # for i, value in enumerate(range_list):

    #     if range_list[i-1] < rand_num and rand_num <= value:
    #         return population[i-1]

    # chromosome = population[3]
    # print("The Chromo Selected has fitness: ", chromosome.fitness)
    return chromosome

In [78]:
def cross_Over(population, list_best):
    # print("Crossover")

    #parent_a - 1 2 3 4 5 6
    #parent_b - 1 2 3 4 5 6 7 8
    #parent_b - 1 2 3 4
    crossed_population = []
    global STUDENT_GROUPS_GLOBAL
    global COURSES_GLOBAL
    for i, chromo in enumerate(list_best):
        if i < 4:
            crossed_population.append(chromo)
    # dimensions = ["Day", "Session", "Student Group", "Teacher"]
    while len(crossed_population) < len(population):
        random_b = randint(0, len(population)-1)
        parent_b = population[random_b]
        parent_a = roulette_wheel_selection(population)

        if random.random() <= CROSS_PROBABILITY:
            pointer = randint(1, min(len(parent_a.days), len(parent_b.days)))
            # pointer = randint(1, len(parent_b.days))
            # for shift in range(pointer):
            #     if shift < len(parent_a.days) and len(parent_b.days):
            #         parent_a.days[shift], parent_b.days[shift] = parent_b.days[shift], parent_a.days[shift]
            #     elif shift < len(parent_a.days):
            #         temp_day = parent_a.days[shift]
            #         parent_a.days.remove(temp_day)
            #         parent_a.days_count -= 1
            #         parent_b.add_Day(temp_day)
            #     elif shift < len(parent_b.days):
            #         temp_day = parent_b.days[shift]
            #         parent_b.days.remove(temp_day)
            #         parent_b.days_count -= 1
            #         parent_a.add_Day(temp_day)

            for shift in range(pointer):
                parent_a.days[shift], parent_b.days[shift] = parent_b.days[shift], parent_a.days[shift]

            if len(crossed_population) < len(population):
                crossed_population.append(parent_a)
            if len(crossed_population) < len(population):
                crossed_population.append(parent_b)
        else:
            if len(list_best) > 0:
                if len(crossed_population) < len(population):
                    crossed_population.append(
                        list_best[randint(0, len(list_best)-1)])
            elif len(crossed_population) < len(population):
                crossed_population.append(parent_a)
        # print("DEEE Random In")
    for i in range(len(crossed_population)):
        crossed_population[i].index = i

    return crossed_population

In [74]:
def mutate(population):

    # mutate_population = []
    global COURSES_GLOBAL
    global STUDENT_GROUPS_GLOBAL
    global TEACHERS_GLOBAL
    global CLASS_IDS_GLOBAL

    for chromo in population:
        if random.random() <= MUTATION_PROBABILITY:
            new_chromo = populate(
                COURSES_GLOBAL, STUDENT_GROUPS_GLOBAL, TEACHERS_GLOBAL, CLASS_IDS_GLOBAL)
            random_day1 = randint(0, new_chromo.days_count-1)
            random_day2 = randint(0, chromo.days_count-1)
            # print("day1:", random_day1)
            # print("day2:", random_day2)
            # print("mutation")
            population[chromo.index].days[random_day2] = new_chromo.days[random_day1]

    return population

In [75]:
def best_Schedule(population):
    # print("Best Schedule")
    best = 0
    chromos = []
    index = 0
    for i, chromo in enumerate(population):
        if chromo.fitness >= best:
            best = chromo.fitness
            index = i
    best_chromo = None
    for chromo in population:
        if chromo.fitness == best:
            chromos.append(chromo)
            if best_chromo == None:
                best_chromo = chromo

    return best_chromo, chromos

In [76]:

def GA(population):
    week_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    print("MAX GENERATIONS: ", MAX_GENERATIONS)
    best_Chromo = None
    list_best = []
    for i in range(MAX_GENERATIONS):

        population = deepcopy(calculate_fitness(population))
        #population = calculate_fitness(population)
        population = deepcopy(cross_Over(population, list_best))
        # population = cross_Over(population, list_best)

        population = deepcopy(mutate(population))
        # population = mutate(population)

        population = deepcopy(calculate_fitness(population))
        #population = calculate_fitness(population)
        temp_best, list_best_temp = best_Schedule(population)
        if best_Chromo == None:
            best_Chromo = temp_best
            list_best = list_best_temp
        if best_Chromo.fitness < temp_best.fitness:
            best_Chromo = temp_best
            list_best = list_best_temp
        # if best_Chromo.fitness < temp_best1.fitness:
        #     best_Chromo = temp_best1
        if i % 50 == 0:
            print("{} iterations passed".format(i))
            print("BEST SOLUTION AFTER {} ITERATIONS: ".format(
                i), best_Chromo.fitness)
            best_courses_list = []
            remaining_Exams = len(
                COURSES_GLOBAL) - best_Chromo.total_exams() + best_Chromo.duplicate_Exams()
            print("Remaining Exams: ", remaining_Exams)
            print("Total Days:", len(best_Chromo.days))
            print("HARD CONSTRAINTS\n", best_Chromo.hard)
            print("SOFT CONSTRAINTS\n", best_Chromo.soft)
            for j, day in enumerate(best_Chromo.days):
                print("Day {}...\nSession 1:".format(week_days[(j%5)]))
                print(day.sessions[0].courseIDs)
                best_courses_list.append(day.sessions[0].courseIDs)
                best_courses_list.append(day.sessions[1].courseIDs)
                print(day.sessions[0].teachers)
                print("Student_Overlap: ",
                      day.sessions[0].overlapping_Students())

                print("Session 2:")
                print(day.sessions[1].courseIDs)
                print(day.sessions[1].teachers)
                print("Student_Overlap: ",
                      day.sessions[1].overlapping_Students())

            best_courses = []
            for list_exam in best_courses_list:
                for exam in list_exam:
                    best_courses.append(exam)
            print("Courses Remaining: ", set(
                COURSES_GLOBAL) - set(best_courses))
            print("Best Courses: ", sorted(best_courses))
            print("Total: ", best_Chromo.total_exams())

    
    print("BEST SOLUTION FOUND WITH FITNESS: ", best_Chromo.fitness)
    print("And duplicate exams: ", best_Chromo.duplicate_Exams())
    print("Total Days:", len(best_Chromo.days))
    best_courses_list = []
    for i, day in enumerate(best_Chromo.days):
        print("Day {}...\nSession 1:".format(week_days[(i%5)]))
        print(day.sessions[0].courseIDs)
        best_courses_list.append(day.sessions[0].courseIDs)
        best_courses_list.append(day.sessions[1].courseIDs)
        print(day.sessions[0].teachers)
        print("Student_Overlap: ", day.sessions[0].overlapping_Students())
        print("Session 2:")
        print(day.sessions[1].courseIDs)
        print(day.sessions[1].teachers)
        print("Student_Overlap: ", day.sessions[1].overlapping_Students())

    best_courses = []
    for list_exam in best_courses_list:
        for exam in list_exam:
            best_courses.append(exam)
    print("Courses Remaining: ", set(COURSES_GLOBAL) - set(best_courses))
    print("Best Courses: ", sorted(best_courses))
    return best_Chromo


In [15]:
main()

', 'SS118']
['Farah Naz', 'Mehreen Alam']
Student_Overlap:  0
Session 2:
['CY2012']
['Nagina Safdar']
Student_Overlap:  0
Day 2...
Session 1:
['EE227', 'CS307']
['Bilal Khalid', 'Amna Irum']
Student_Overlap:  2
Session 2:
['SE110']
['Usman Ashraf']
Student_Overlap:  0
Day 3...
Session 1:
['SS111', 'CS219']
['Sara Aziz', 'Naveed Ahmad']
Student_Overlap:  1
Session 2:
['CS218']
['Asif Naeem']
Student_Overlap:  0
Day 4...
Session 1:
['CS328', 'CS118']
['Behjat Zuhaira', 'Waqas Munir']
Student_Overlap:  8
Session 2:
['MG223']
['Zohaib Iqbal']
Student_Overlap:  0
Day 5...
Session 1:
['SS113', 'MT205']
['Noor ul Ain', 'Sanaa Ilyas']
Student_Overlap:  6
Session 2:
['CS302']
['Mehboobullah']
Student_Overlap:  0
Day 6...
Session 1:
['MT224', 'EE229']
['Mehreen Alam', 'Zainab Moin']
Student_Overlap:  5
Session 2:
['CS211']
['Mehwish Hassan']
Student_Overlap:  0
Day 7...
Session 1:
['CS217', 'MG220']
['Tayyaba Zainab', 'Adnan Tariq']
Student_Overlap:  7
Session 2:
['SS152']
['Hamda Khan']
Student

In [52]:
main()  # 5 population - 2000 iterations

'DS3011']
['Sehrish Hassan', 'Mehreen Alam']
Student_Overlap:  5
Day Monday...
Session 1:
['CS118', 'CS328']
['Gul e Aisha', 'Subhan Ullah']
Student_Overlap:  8
Session 2:
['MG220']
['Zohaib Iqbal']
Student_Overlap:  0
Courses Remaining:  set()
Best Courses:  ['AI2011', 'CS118', 'CS211', 'CS217', 'CS218', 'CS219', 'CS220', 'CS302', 'CS307', 'CS328', 'CY2012', 'DS3011', 'EE227', 'EE229', 'MG220', 'MG223', 'MT205', 'MT224', 'SE110', 'SS111', 'SS113', 'SS118', 'SS152']
Total:  23
1500 iterations passed
BEST SOLUTION AFTER 1500 ITERATIONS:  99323.0
Remaining Exams:  0
Total Days: 6
HARD CONSTRAINTS
 {'Students Overlap': 'Violated', 'Teachers Overlap': 'Not Violated', 'Consecutive Teachers': 'Not Violated', 'Duplicate Course': 'Not Violated', 'Weekend': 'Not Violated', 'All Courses': 'Not Violated'}
SOFT CONSTRAINTS
 {'Break on Friday': 'Not Violated', 'Consecutive Exam': 'Violated', 'MG before CS': 'Violated', 'Faculty Meeting': 'Not Violated'}
Day Monday...
Session 1:
['CS307', 'CS218']
[

In [27]:
main()

day...
Session 1:
['MG223', 'MT205']
['Sumera Abbas', 'Asma Nisa']
Student_Overlap:  5
Session 2:
['MG220']
['Kifayat Ullah']
Student_Overlap:  0
Day Wednesday...
Session 1:
['CY2012', 'EE229']
['Sadia Nauman', 'Mehboobullah']
Student_Overlap:  4
Session 2:
['SS111']
['Kifayat Ullah']
Student_Overlap:  0
Courses Remaining:  {'SS152'}
Best Courses:  ['AI2011', 'CS118', 'CS211', 'CS217', 'CS218', 'CS219', 'CS219', 'CS220', 'CS302', 'CS307', 'CS328', 'CY2012', 'DS3011', 'EE227', 'EE229', 'MG220', 'MG223', 'MT205', 'MT224', 'SE110', 'SS111', 'SS113', 'SS118']
Total:  23
1550 iterations passed
BEST SOLUTION AFTER 1550 ITERATIONS:  99911.0
Remaining Exams:  1
Total Days: 8
HARD CONSTRAINTS
 {'Students Overlap': 'Not Violated', 'Teachers Overlap': 'Violated', 'Consecutive Teachers': 'Not Violated', 'Duplicate Course': 'Violated', 'Weekend': 'Not Violated', 'All Courses': 'Violated'}
SOFT CONSTRAINTS
 {'Break on Friday': 'Not Violated', 'Consecutive Exam': 'Violated', 'MG before CS': 'Violated

In [59]:
main()

000000100100101010100001010110111001001000100000000100101111101111000101010101100000001010011110011010100101000001010101000100000001001010110111111011001110000000000010110110100010110100111011110010010010000000010010100001000110101100010000000000110001110010111100000010100000000011000101000110000


CHROMO 45 = 101101000001000101001001001101010001010101011001011101100001100101101001101101110001


CHROMO 46 = 101110000010000101011000011001100000000111000000111101001001111000000010010001010101110100010110000000001100111100000101010001000000000101001010010010100111000010000000011000010011101010001001100000001011010100100011000101100000000000111110101111101010001011000000011000111001010010011010100100000001111101011001000111000000000000110101110001001001010100001000000011011111101100000


CHROMO 47 = 101111000001000101001001001101010001010101011001011101100001100101101001101101110001


CHROMO 48 = 1100000000000111001101110110000010110000110000000010010111011010110001101000100000000100101001

In [84]:
main()

dent_Overlap:  5
Session 2:
['SS152']
['Hasan Mujtaba']
Student_Overlap:  0
Day Tuesday...
Session 1:
['SS111', 'MT224']
['Subhan Ullah', 'Hassan Mustafa']
Student_Overlap:  4
Session 2:
['EE229']
['Shehreyar Rashid']
Student_Overlap:  0
Day Wednesday...
Session 1:
['CY2012']
['Hassan Raza']
Student_Overlap:  0
Session 2:
['CS219']
['Noor ul Ain']
Student_Overlap:  0
Courses Remaining:  set()
Best Courses:  ['AI2011', 'CS118', 'CS211', 'CS217', 'CS218', 'CS219', 'CS220', 'CS302', 'CS307', 'CS328', 'CY2012', 'DS3011', 'EE227', 'EE229', 'MG220', 'MG223', 'MT205', 'MT224', 'SE110', 'SS111', 'SS113', 'SS118', 'SS152']
Total:  23
50 iterations passed
BEST SOLUTION AFTER 50 ITERATIONS:  99839.0
Remaining Exams:  0
Total Days: 8
HARD CONSTRAINTS
 {'Students Overlap': 'Violated', 'Teachers Overlap': 'Not Violated', 'Consecutive Teachers': 'Not Violated', 'Duplicate Course': 'Not Violated', 'Weekend': 'Not Violated', 'All Courses': 'Not Violated'}
SOFT CONSTRAINTS
 {'Break on Friday': 'Not Viol

In [85]:
main()

Shehreyar Rashid']
Student_Overlap:  0
Day Tuesday...
Session 1:
['SS111', 'MT224']
['Farwa Batool', 'Ameen Chilwan']
Student_Overlap:  4
Session 2:
['SS152']
['Mehwish Hassan']
Student_Overlap:  0
Day Wednesday...
Session 1:
['MT205']
['Waqas Munir']
Student_Overlap:  0
Session 2:
['CS211']
['Noreen Jamil']
Student_Overlap:  0
Courses Remaining:  set()
Best Courses:  ['AI2011', 'CS118', 'CS211', 'CS217', 'CS218', 'CS219', 'CS220', 'CS302', 'CS307', 'CS328', 'CY2012', 'DS3011', 'EE227', 'EE229', 'MG220', 'MG223', 'MT205', 'MT224', 'SE110', 'SS111', 'SS113', 'SS118', 'SS152']
Total:  23
50 iterations passed
BEST SOLUTION AFTER 50 ITERATIONS:  99839.0
Remaining Exams:  0
Total Days: 8
HARD CONSTRAINTS
 {'Students Overlap': 'Violated', 'Teachers Overlap': 'Not Violated', 'Consecutive Teachers': 'Not Violated', 'Duplicate Course': 'Not Violated', 'Weekend': 'Not Violated', 'All Courses': 'Not Violated'}
SOFT CONSTRAINTS
 {'Break on Friday': 'Not Violated', 'Consecutive Exam': 'Violated', '